<a href="https://colab.research.google.com/github/Saumil-Agarwal/Aircraft_Accident-Hackerearth/blob/master/Log_facies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting and Loading Dataset

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [0]:
df = pd.read_csv('CAX_LogFacies_Train_File.csv')
print(df.head())
print(df.shape)
print(df.describe())

   row_id  well_id          GR  label
0       0        0  143.510000      0
1       1        0  112.790928      0
2       2        0  123.531856      0
3       3        0  111.692784      0
4       4        0  123.613712      0
(4400000, 4)
             row_id       well_id            GR         label
count  4.400000e+06  4.400000e+06  4.400000e+06  4.400000e+06
mean   5.495000e+02  1.999500e+03  1.092733e+02  1.184489e+00
std    3.175426e+02  1.154701e+03  3.424240e+01  1.465016e+00
min    0.000000e+00  0.000000e+00 -3.181915e+00  0.000000e+00
25%    2.747500e+02  9.997500e+02  8.179222e+01  0.000000e+00
50%    5.495000e+02  1.999500e+03  1.203849e+02  0.000000e+00
75%    8.242500e+02  2.999250e+03  1.347291e+02  2.000000e+00
max    1.099000e+03  3.999000e+03  1.970614e+02  4.000000e+00


In [0]:
df['label'].value_counts()

0    2313281
2     525376
3     522140
1     520744
4     518459
Name: label, dtype: int64

# Random Forest Classifier

In [0]:
X = df['GR']
y = df['label']

X_tr, X_tst, y_tr, y_tst = train_test_split(X, y, test_size=0.3)



In [0]:
y_tst.value_counts()

0    693819
2    157656
3    156537
1    156361
4    155627
Name: label, dtype: int64

In [0]:
X = df.loc[0:1000000, ['GR']] 
y = df.loc[0:1000000, ['label']] 

X_tr, X_tst, y_tr, y_tst = train_test_split(X, y, test_size=0.15)

X_tr = X_tr.values
X_tst = X_tst.values

y_tr = y_tr.values
y_tst = y_tst.values



In [0]:

clf=RandomForestClassifier(n_estimators=100,min_samples_leaf=3)
clf.fit(X_tr, y_tr)
print('fit done')
predicted= clf.predict(X_tst)
print('prediction done')
print(metrics.accuracy_score(y_tst, predicted))
# print(RandomForestClassifier())

In [0]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_tst, predicted))

# LSTM

In [0]:
df.isnull().values.any()

False

In [0]:
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Input, SimpleRNN, LSTM
# from keras.activation import relu
# from keras.optimizer import adam
from sklearn import preprocessing
import sys

In [0]:
df.head(1101)

,row_id,well_id,GR,label
0,0,0,143.510000,0
1,1,0,112.790928,0
2,2,0,123.531856,0
3,3,0,111.692784,0
4,4,0,123.613712,0
...,...,...,...,...
1096,1096,0,158.257216,0
1097,1097,0,139.508144,0
1098,1098,0,130.589072,0
1099,1099,0,154.930000,0


In [0]:
# X = df[[GR']]
# y = df[['label']]

X = df.loc[:, ['row_id','GR']] 
y = df.loc[:, ['label']]

In [0]:
# pd.set_option('display.max_rows', 1100)
# np.set_printoptions(threshold=1000)

In [0]:
X = df.loc[:, ['row_id','GR']]
X_changed = preprocessing.scale(X)  #normalizing X
print(X_changed)
# X_changed = X.values
X_changed = X_changed.reshape(-1,1100,2) #changing gr as parameters
print('Reshape')
print(X_changed)
print(X_changed.shape)

# y_changed = y.values
# y_changed = y_changed.reshape(-1,1100)
# print(y_changed)
# print(y_changed.shape)

In [0]:
y = y.values  #One hot encoding
onehotencoder = preprocessing.OneHotEncoder(categories = [[0,1,2,3,4]])  #hot encoding y
y_one_hot = onehotencoder.fit_transform(y.reshape(-1,1)).toarray()
print(y_one_hot)

In [0]:
print(y_one_hot.shape)

(4400000, 5)


In [0]:
y_changed = y_one_hot.reshape(-1,1100,5)  # changing the outputs so that each CELL of lstm gives an output
print(y_changed)
print(y_changed.shape)

In [0]:
X_tr, X_tst, y_tr, y_tst = train_test_split(X_changed, y_changed, test_size=0.2)
print(X_tr.shape)
print(X_tst.shape)

X_tr = np.reshape(X_tr, (X_tr.shape[0], X_tr.shape[1],1)) # reshaping the X to pass as LSTM input
X_tst = np.reshape(X_tst, (X_tst.shape[0], X_tst.shape[1],1))
print(X_tr.shape)
print(X_tst.shape)
print(y_tr.shape)
print(y_tst.shape)
print(X_tr)

**Implementing various Lstm variations
These have been implemented in hardcoded fashion and not through function so that they can be run overnight on colab**

**There was leak in dataset - repeating pattern of digits after the decimal point. Hence, the results of applied lstm weren't effective and accurate enough.**

## LSTM-1

In [0]:
# define the keras model

batch_input_shape = (None,1100,2)  #(no_of_inputs,length_of_input_sequences,length of each vector)

model = Sequential()
model.add(LSTM(units = 1, return_sequences=True, batch_input_shape=batch_input_shape))
model.add(LSTM(units = 5, return_sequences=True,recurrent_activation='softmax'))

# model.add(LSTM(4, return_sequences=True,return_state = True, input_shape=(1000 ,1)))
# model.add(LSTM(8, return_sequences=True))
# model.add(Dropout(0.5))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(32, activation='relu'))
# model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Epoch 100/100 loss: nan - acc: 0.5261 without row id
#Accuracy is: 52.41375

In [0]:
model.summary()

In [0]:
model.fit(X_tr,y_tr, epochs= 100, batch_size=256)     #validation_data ?

Epoch 1/100
3200/3200 [==============================] - 39s 12ms/step - loss: 4.3867 - acc: 0.1667
Epoch 2/100
3200/3200 [==============================] - 38s 12ms/step - loss: 4.1517 - acc: 0.3513
Epoch 3/100
3200/3200 [==============================] - 38s 12ms/step - loss: 4.8503 - acc: 0.3311
Epoch 4/100
3200/3200 [==============================] - 38s 12ms/step - loss: 6.8904 - acc: 0.1344
Epoch 5/100
3200/3200 [==============================] - 38s 12ms/step - loss: 5.4108 - acc: 0.1172
Epoch 6/100
3200/3200 [==============================] - 38s 12ms/step - loss: 3.0524 - acc: 0.1179
Epoch 7/100
3200/3200 [==============================] - 38s 12ms/step - loss: 2.8033 - acc: 0.1179
Epoch 8/100
3200/3200 [==============================] - 38s 12ms/step - loss: 2.5509 - acc: 0.1178
Epoch 9/100
3200/3200 [==============================] - 38s 12ms/step - loss: 2.2583 - acc: 0.0961
Epoch 10/100
 512/3200 [===>..........................] - ETA: 30s - loss: 2.2169 - acc: 0.0937

KeyboardInterrupt: ignored

In [0]:
# Make Predictions
y_pred = model.predict(X_tst)
print(y_pred.shape)



#Converting predictions to label
pred = []
for i in range(y_pred.shape[0]):
  for j in range(y_pred.shape[1]):
      pred.append(np.argmax(y_pred[i][j]))

print(len(pred))
#Converting one hot encoded test label to label
test = []
for i in range(y_tst.shape[0]):
  for j in range(y_tst.shape[1]):
    test.append(np.argmax(y_tst[i][j]))

print(len(test))
from sklearn.metrics import accuracy_score
a = accuracy_score(np.asarray(pred),np.asarray(test))
print('Accuracy is:', a*100)

(800, 1100, 5)
880000
880000
Accuracy is: 52.41375


## LSTM-2

In [0]:
# define the keras model - Thisonegoood.......

batch_input_shape = (None,1100,2)  #(no_of_inputs,length_of_input_sequences,length of each vector)

model = Sequential()
model.add(LSTM(units = 1, return_sequences=True, batch_input_shape=batch_input_shape))
model.add(LSTM(units = 1, return_sequences=True))
model.add(LSTM(units = 5, return_sequences=True))

# model.add(LSTM(4, return_sequences=True,return_state = True, input_shape=(1000 ,1)))
# model.add(LSTM(8, return_sequences=True))
# model.add(Dropout(0.5))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(32, activation='relu'))
# model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Epoch 3/100 loss: 2.6473 - acc: 0.5358  without row id
#Epoch 100/100 loss: 1.1276 - acc: 0.0799
#Accuracy is: 7.985681818181818

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 1100, 1)           16        
_________________________________________________________________
lstm_5 (LSTM)                (None, 1100, 1)           12        
_________________________________________________________________
lstm_6 (LSTM)                (None, 1100, 5)           140       
Total params: 168
Trainable params: 168
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_tr,y_tr, epochs= 150, batch_size=128)     #validation_data ?

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/150





3200/3200 [==============================] - 53s 17ms/step - loss: 1.5789 - acc: 0.2768
Epoch 2/150
3200/3200 [==============================] - 51s 16ms/step - loss: 1.4995 - acc: 0.3425
Epoch 3/150
3200/3200 [==============================] - 52s 16ms/step - loss: 1.4586 - acc: 0.3404
Epoch 4/150
3200/3200 [==============================] - 51s 16ms/step - loss: 1.4895 - acc: 0.3355
Epoch 5/150
3200/3200 [==============================] - 50s 16ms/step - loss: 1.4358 - acc: 0.3195
Epoch 6/150
3200/3200 [==============================] - 52s 16ms/step - loss: 1.4184 - acc: 0.3203
Epoch 7/150
3200/3200 [==============================] - 51s 16ms/step - loss: 1.4055 - acc: 0.3206
Epoch 8/150
3200/3200 [==============================] - 51s 16ms/step - loss: 1.4003 - acc: 0.3309
Epoch 9/150
3200/3200 [==============================] - 51s 16ms/step - loss: 1.3908 - acc: 0.332

In [0]:
model.fit(X_tr,y_tr, epochs= 50, batch_size=128)     #validation_data ?

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.2,
                              patience=5, min_lr=0.001)
model.fit(X_train, Y_train, callbacks=[reduce_lr])
model.fit(X_tr,y_tr, epochs= 120, batch_size=128)     #validation_data ?

In [0]:
# Make Predictions
y_pred = model.predict(X_tst)
print(y_pred.shape)



#Converting predictions to label
pred = []
for i in range(y_pred.shape[0]):
  for j in range(y_pred.shape[1]):
      pred.append(np.argmax(y_pred[i][j]))

print(len(pred))
#Converting one hot encoded test label to label
test = []
for i in range(y_tst.shape[0]):
  for j in range(y_tst.shape[1]):
    test.append(np.argmax(y_tst[i][j]))

print(len(test))
from sklearn.metrics import accuracy_score
a = accuracy_score(np.asarray(pred),np.asarray(test))
print('Accuracy is:', a*100)

## LSTM-3

In [0]:
# define the keras model

batch_input_shape = (None,1100,2)  #(no_of_inputs,length_of_input_sequences,length of each vector)

model = Sequential()
model.add(LSTM(units = 5, return_sequences=True, batch_input_shape=batch_input_shape,recurrent_activation='sigmoid'))


# model.add(LSTM(4, return_sequences=True,return_state = True, input_shape=(1000 ,1)))
# model.add(LSTM(8, return_sequences=True))
# model.add(Dropout(0.5))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(32, activation='relu'))
# model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Epoch 100/100 loss: 1.2052 - acc: 0.1216 without row id
#Accuracy is: 12.273068181818182

In [0]:
model.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_30 (LSTM)               (None, 1100, 5)           140       
Total params: 140
Trainable params: 140
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_tr,y_tr, epochs= 100, batch_size=256)     #validation_data ?

Epoch 1/100
3200/3200 [==============================] - 8s 3ms/step - loss: 1.8476 - acc: 0.0819
Epoch 2/100
3200/3200 [==============================] - 6s 2ms/step - loss: 1.7626 - acc: 0.0808
Epoch 3/100
3200/3200 [==============================] - 6s 2ms/step - loss: 1.6883 - acc: 0.0792
Epoch 4/100
3200/3200 [==============================] - 6s 2ms/step - loss: 1.6275 - acc: 0.0777
Epoch 5/100
3200/3200 [==============================] - 6s 2ms/step - loss: 1.5739 - acc: 0.0759
Epoch 6/100
3200/3200 [==============================] - 6s 2ms/step - loss: 1.5253 - acc: 0.0737
Epoch 7/100
3200/3200 [==============================] - 6s 2ms/step - loss: 1.4802 - acc: 0.0727
Epoch 8/100
3200/3200 [==============================] - 6s 2ms/step - loss: 1.4421 - acc: 0.0732
Epoch 9/100
3200/3200 [==============================] - 6s 2ms/step - loss: 1.4063 - acc: 0.0751
Epoch 10/100
3200/3200 [==============================] - 6s 2ms/step - loss: 1.3627 - acc: 0.0800
Epoch 11/100
3200/3

In [0]:
# Make Predictions
y_pred = model.predict(X_tst)
print(y_pred.shape)



#Converting predictions to label
pred = []
for i in range(y_pred.shape[0]):
  for j in range(y_pred.shape[1]):
      pred.append(np.argmax(y_pred[i][j]))

print(len(pred))
#Converting one hot encoded test label to label
test = []
for i in range(y_tst.shape[0]):
  for j in range(y_tst.shape[1]):
    test.append(np.argmax(y_tst[i][j]))

print(len(test))
from sklearn.metrics import accuracy_score
a = accuracy_score(np.asarray(pred),np.asarray(test))
print('Accuracy is:', a*100)

## LSTM-4

In [0]:
# define the keras model

batch_input_shape = (None,1100,1)  #(no_of_inputs,length_of_input_sequences,length of each vector)

model = Sequential()
model.add(LSTM(units = 1, return_sequences=True, batch_input_shape=batch_input_shape))
model.add(LSTM(units = 5, return_sequences=True))


# model.add(LSTM(4, return_sequences=True,return_state = True, input_shape=(1000 ,1)))
# model.add(LSTM(8, return_sequences=True))
# model.add(Dropout(0.5))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(32, activation='relu'))
# model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Epoch 100/100 loss: nan - acc: 0.5261
# Accuracy is: 52.41375

In [0]:
model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_31 (LSTM)               (None, 1100, 1)           12        
_________________________________________________________________
lstm_32 (LSTM)               (None, 1100, 5)           140       
Total params: 152
Trainable params: 152
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_tr,y_tr, epochs= 100, batch_size=256)     #validation_data ?

Epoch 1/100
3200/3200 [==============================] - 15s 5ms/step - loss: 9.8114 - acc: 0.4283
Epoch 2/100
3200/3200 [==============================] - 12s 4ms/step - loss: 5.5147 - acc: 0.0913
Epoch 3/100
3200/3200 [==============================] - 12s 4ms/step - loss: 4.9170 - acc: 0.0661
Epoch 4/100
3200/3200 [==============================] - 12s 4ms/step - loss: 4.3889 - acc: 0.1044
Epoch 5/100
3200/3200 [==============================] - 12s 4ms/step - loss: 4.0774 - acc: 0.1200
Epoch 6/100
3200/3200 [==============================] - 12s 4ms/step - loss: 3.8093 - acc: 0.1200
Epoch 7/100
3200/3200 [==============================] - 12s 4ms/step - loss: 3.4682 - acc: 0.1200
Epoch 8/100
3200/3200 [==============================] - 12s 4ms/step - loss: 3.2867 - acc: 0.1200
Epoch 9/100
3200/3200 [==============================] - 12s 4ms/step - loss: 3.2600 - acc: 0.1200
Epoch 10/100
3200/3200 [==============================] - 12s 4ms/step - loss: 3.2896 - acc: 0.1200
Epoch 11/

In [0]:
# Make Predictions
y_pred = model.predict(X_tst)
print(y_pred.shape)



#Converting predictions to label
pred = []
for i in range(y_pred.shape[0]):
  for j in range(y_pred.shape[1]):
      pred.append(np.argmax(y_pred[i][j]))

print(len(pred))
#Converting one hot encoded test label to label
test = []
for i in range(y_tst.shape[0]):
  for j in range(y_tst.shape[1]):
    test.append(np.argmax(y_tst[i][j]))

print(len(test))
from sklearn.metrics import accuracy_score
a = accuracy_score(np.asarray(pred),np.asarray(test))
print('Accuracy is:', a*100)

(800, 1100, 5)
880000
880000
Accuracy is: 52.41375


# Adding a new feature for LSTM(difference of GR Values)

In [0]:
X = df.loc[:, ['GR']] 
y = df.loc[:, ['label']]

In [0]:
X=X.values
print(X.shape)
print(X)

In [0]:
X1 = np.delete(X, (X.shape[0]-1), axis=0)
X1 = np.insert(X1,0,np.array([0]),axis=0)
print(X1)
print(X1.shape)

In [0]:
X2 = X-X1
print(X2)
X2 = np.delete(X2, (0), axis=0)
X2 = np.insert(X2,0,np.array([0]),axis=0)         #Here assigned the first value as the origin.Need to change it later.
                                                  #Also for each well it should be different need tochange that too
print(X2)
print(X2.shape)

In [0]:
X = df.loc[:, ['GR']]
X_changed = preprocessing.scale(X2)  #normalizing X2
# X_changed = X.values
X_changed = X_changed.reshape(-1,1100) #changing gr as parameters
print(X_changed)
print(X_changed.shape)


In [0]:
y = y.values
onehotencoder = preprocessing.OneHotEncoder(categories = [[0,1,2,3,4]])  #hot encoding y
y_one_hot = onehotencoder.fit_transform(y.reshape(-1,1)).toarray()
print(y_one_hot)
print(y_one_hot.shape)

In [0]:
y_changed = y_one_hot.reshape(-1,1100,5)  # changing the outputs so that each CELL of lstm gives an output
print(y_changed)
print(y_changed.shape)

In [0]:
X_tr, X_tst, y_tr, y_tst = train_test_split(X_changed, y_changed, test_size=0.2)
print(X_tr.shape)
print(X_tst.shape)

X_tr = np.reshape(X_tr, (X_tr.shape[0], X_tr.shape[1],1)) # reshaping the X to pass as LSTM input
X_tst = np.reshape(X_tst, (X_tst.shape[0], X_tst.shape[1],1))
print(X_tr.shape)
print(X_tst.shape)
print(y_tr.shape)
print(y_tst.shape)
print(X_tr)

## LSTM-5

In [0]:
# define the keras model

batch_input_shape = (None,1100,1)  #(no_of_inputs,length_of_input_sequences,length of each vector)

model = Sequential()
model.add(LSTM(units = 1, return_sequences=True, batch_input_shape=batch_input_shape,recurrent_activation='sigmoid'))
model.add(LSTM(units = 5, return_sequences=True, batch_input_shape=batch_input_shape,recurrent_activation='softmax'))

# model.add(LSTM(4, return_sequences=True,return_state = True, input_shape=(1000 ,1)))
# model.add(LSTM(8, return_sequences=True))
# model.add(Dropout(0.5))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(32, activation='relu'))
# model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1100, 1)           12        
_________________________________________________________________
lstm_2 (LSTM)                (None, 1100, 5)           140       
Total params: 152
Trainable params: 152
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_tr,y_tr, epochs= 200, batch_size=256)     #validation_data ?

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





3200/3200 [==============================] - 15s 5ms/step - loss: 7.6598 - acc: 0.3640
Epoch 2/100
3200/3200 [==============================] - 15s 5ms/step - loss: 3.7469 - acc: 0.3863
Epoch 3/100
3200/3200 [==============================] - 13s 4ms/step - loss: 2.4010 - acc: 0.4104
Epoch 4/100
3200/3200 [==============================] - 13s 4ms/step - loss: 1.8672 - acc: 0.4199
Epoch 5/100
3200/3200 [==============================] - 13s 4ms/step - loss: 1.7343 - acc: 0.4410
Epoch 6/100
3200/3200 [==============================] - 13s 4ms/step - loss: 1.6751 - acc: 0.4647
Epoch 7/100
3200/3200 [==============================] - 13s 4ms/step - loss: 1.6230 - acc: 0.4857
Epoch 8/100
3200/3200 [==============================] - 13s 4ms/step - loss: 1.5729 - acc: 0.5047
Epoch 9/100
3200/3200 [==============================] - 13s 4ms/step - loss: 1.5463 - acc: 0.5105
Epoch 1

In [0]:
# Make Predictions
y_pred = model.predict(X_tst)
print(y_pred.shape)



#Converting predictions to label
pred = []
for i in range(y_pred.shape[0]):
  for j in range(y_pred.shape[1]):
      pred.append(np.argmax(y_pred[i][j]))

print(len(pred))
#Converting one hot encoded test label to label
test = []
for i in range(y_tst.shape[0]):
  for j in range(y_tst.shape[1]):
    test.append(np.argmax(y_tst[i][j]))

print(len(test))
from sklearn.metrics import accuracy_score
a = accuracy_score(np.asarray(pred),np.asarray(test))
print('Accuracy is:', a*100)

(800, 1100, 5)
880000
880000
Accuracy is: 52.81795454545455


## LSTM-6

In [0]:
# define the keras model

batch_input_shape = (None,1100,1)  #(no_of_inputs,length_of_input_sequences,length of each vector)

model = Sequential()
model.add(LSTM(units = 5, return_sequences=True, batch_input_shape=batch_input_shape,recurrent_activation='sigmoid'))

# model.add(LSTM(4, return_sequences=True,return_state = True, input_shape=(1000 ,1)))
# model.add(LSTM(8, return_sequences=True))
# model.add(Dropout(0.5))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(32, activation='relu'))
# model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 1100, 5)           140       
Total params: 140
Trainable params: 140
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_tr,y_tr, epochs= 150, batch_size=64)     #validation_data ?

Epoch 1/150
3200/3200 [==============================] - 64s 20ms/step - loss: 4.1542 - acc: 0.1697
Epoch 2/150
3200/3200 [==============================] - 62s 19ms/step - loss: 3.5006 - acc: 0.2719
Epoch 3/150
3200/3200 [==============================] - 62s 20ms/step - loss: 3.1557 - acc: 0.3234
Epoch 4/150
3200/3200 [==============================] - 63s 20ms/step - loss: 2.6420 - acc: 0.3716
Epoch 5/150
3200/3200 [==============================] - 61s 19ms/step - loss: 2.1690 - acc: 0.4295
Epoch 6/150
3200/3200 [==============================] - 62s 19ms/step - loss: 1.7099 - acc: 0.3732
Epoch 7/150
3200/3200 [==============================] - 62s 19ms/step - loss: 1.5365 - acc: 0.4218
Epoch 8/150
3200/3200 [==============================] - 62s 19ms/step - loss: 1.4937 - acc: 0.4922
Epoch 9/150
3200/3200 [==============================] - 63s 20ms/step - loss: 1.4742 - acc: 0.5138
Epoch 10/150
3200/3200 [==============================] - 62s 19ms/step - loss: 1.4610 - acc: 0.5194

KeyboardInterrupt: ignored

In [0]:
# Make Predictions
y_pred = model.predict(X_tst)
print(y_pred.shape)



#Converting predictions to label
pred = []
for i in range(y_pred.shape[0]):
  for j in range(y_pred.shape[1]):
      pred.append(np.argmax(y_pred[i][j]))

print(len(pred))
#Converting one hot encoded test label to label
test = []
for i in range(y_tst.shape[0]):
  for j in range(y_tst.shape[1]):
    test.append(np.argmax(y_tst[i][j]))

print(len(test))
from sklearn.metrics import accuracy_score
a = accuracy_score(np.asarray(pred),np.asarray(test))
print('Accuracy is:', a*100)

(800, 1100, 5)
880000
880000
Accuracy is: 52.25863636363637


## LSTM-7

In [0]:
# define the keras model

batch_input_shape = (None,1100,1)  #(no_of_inputs,length_of_input_sequences,length of each vector)

model = Sequential()
model.add(LSTM(units = 1, return_sequences=True, batch_input_shape=batch_input_shape,recurrent_activation='sigmoid'))
model.add(LSTM(units = 1, return_sequences=True, batch_input_shape=batch_input_shape,recurrent_activation='sigmoid'))
model.add(LSTM(units = 5, return_sequences=True, batch_input_shape=batch_input_shape,recurrent_activation='softmax'))

# model.add(LSTM(4, return_sequences=True,return_state = True, input_shape=(1000 ,1)))
# model.add(LSTM(8, return_sequences=True))
# model.add(Dropout(0.5))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(32, activation='relu'))
# model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 1100, 1)           12        
_________________________________________________________________
lstm_7 (LSTM)                (None, 1100, 1)           12        
_________________________________________________________________
lstm_8 (LSTM)                (None, 1100, 5)           140       
Total params: 164
Trainable params: 164
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_tr,y_tr, epochs= 150, batch_size=64)     #validation_data ?

Epoch 1/150
3200/3200 [==============================] - 201s 63ms/step - loss: 0.1371 - acc: 0.5185
Epoch 2/150
3200/3200 [==============================] - 200s 63ms/step - loss: 1.1921e-07 - acc: 0.5265
Epoch 3/150
3200/3200 [==============================] - 201s 63ms/step - loss: 1.1921e-07 - acc: 0.5265
Epoch 4/150
3200/3200 [==============================] - 199s 62ms/step - loss: 1.1921e-07 - acc: 0.5265
Epoch 5/150
3200/3200 [==============================] - 199s 62ms/step - loss: 1.1921e-07 - acc: 0.5265
Epoch 6/150
3200/3200 [==============================] - 202s 63ms/step - loss: 1.1921e-07 - acc: 0.5265
Epoch 7/150
3200/3200 [==============================] - 202s 63ms/step - loss: 1.1921e-07 - acc: 0.5265
Epoch 8/150
3200/3200 [==============================] - 201s 63ms/step - loss: 1.1921e-07 - acc: 0.5265
Epoch 9/150
3200/3200 [==============================] - 200s 63ms/step - loss: 1.1921e-07 - acc: 0.5265
Epoch 10/150
3200/3200 [==============================] - 2

KeyboardInterrupt: ignored

In [0]:
# Make Predictions
y_pred = model.predict(X_tst)
print(y_pred.shape)



#Converting predictions to label
pred = []
for i in range(y_pred.shape[0]):
  for j in range(y_pred.shape[1]):
      pred.append(np.argmax(y_pred[i][j]))

print(len(pred))
#Converting one hot encoded test label to label
test = []
for i in range(y_tst.shape[0]):
  for j in range(y_tst.shape[1]):
    test.append(np.argmax(y_tst[i][j]))

print(len(test))
from sklearn.metrics import accuracy_score
a = accuracy_score(np.asarray(pred),np.asarray(test))
print('Accuracy is:', a*100)

(800, 1100, 5)
880000
880000
Accuracy is: 52.28363636363637


# Adding diff as a feature(same as used for LSTM)

In [0]:
X = df.loc[:, ['GR']] 
y = df.loc[:, ['label']]

In [0]:
X=X.values
print(X.shape)
print(X)

(4400000, 1)
[[143.51      ]
 [112.79092812]
 [123.53185623]
 ...
 [142.73450409]
 [140.03725205]
 [152.4       ]]


In [0]:
X1 = np.delete(X, (X.shape[0]-1), axis=0)
X1 = np.insert(X1,0,np.array([0]),axis=0)
print(X1)
print(X1.shape)

[[  0.        ]
 [143.51      ]
 [112.79092812]
 ...
 [142.63175614]
 [142.73450409]
 [140.03725205]]
(4400000, 1)


In [0]:
X2 = X-X1
print(X2)
X2 = np.delete(X2, (0), axis=0)
X2 = np.insert(X2,0,np.array([0]),axis=0)         #Here assigned the first value as the origin.Need to change it later.
                                                  #Also for each well it should be different need tochange that too
print(X2)
print(X2.shape)

[[ 1.43510000e+02]
 [-3.07190719e+01]
 [ 1.07409281e+01]
 ...
 [ 1.02747953e-01]
 [-2.69725205e+00]
 [ 1.23627480e+01]]
[[  0.        ]
 [-30.71907188]
 [ 10.74092812]
 ...
 [  0.10274795]
 [ -2.69725205]
 [ 12.36274795]]
(4400000, 1)


In [0]:
 diff = pd.DataFrame(data= X2,columns=['Diff'])  # 1st row as the column names
 diff.head()

,Diff
0,0.000000
1,-30.719072
2,10.740928
3,-11.839072
4,11.920928


In [0]:
df.head()

,row_id,well_id,GR,label
0,0,0,143.510000,0
1,1,0,112.790928,0
2,2,0,123.531856,0
3,3,0,111.692784,0
4,4,0,123.613712,0


In [0]:
result = pd.concat([df[['row_id','well_id','GR']], diff,df['label']], axis=1)
result.head(2200)

,row_id,well_id,GR,Diff,label
0,0,0,143.510000,0.000000,0
1,1,0,112.790928,-30.719072,0
2,2,0,123.531856,10.740928,0
3,3,0,111.692784,-11.839072,0
4,4,0,123.613712,11.920928,0
...,...,...,...,...,...
2195,1095,1,158.788972,-0.867243,0
2196,1096,1,139.421729,-19.367243,0
2197,1097,1,157.724486,18.302757,0
2198,1098,1,159.127243,1.402757,0


In [0]:
X = result[['row_id','well_id','GR','Diff']]
y= result[['label']]

In [0]:
from sklearn.decomposition import PCA
for i in range(1,4): 
  pca = PCA(n_components = i)
  pca.fit(X)
  print(sum(pca.explained_variance_ratio_))

0.9287684605732245
0.9990314700359433
0.9998356045104132


# Artificial Nueral Network

In [0]:
X = result[['row_id','GR','Diff']]
y= result[['label']]

y = y.values
onehotencoder = preprocessing.OneHotEncoder(categories = [[0,1,2,3,4]])  #hot encoding y
y_one_hot = onehotencoder.fit_transform(y.reshape(-1,1)).toarray()
print(y_one_hot)




In [0]:
X_train,X_test,y_train,y_test = train_test_split(X,y_one_hot,test_size = 0.1)

# define the keras  
model = Sequential()
model.add(Dense(8, input_dim=X.shape[1], activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(15, activation='relu'))
# model.add(Dense(12, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
model.fit(X_tr,y_tr, epochs= 150, batch_size=256)     #validation_data ?

In [0]:
# Make Predictions
y_pred = model.predict(X_tst)
print(y_pred.shape)



#Converting predictions to label
pred = []
for i in range(y_pred.shape[0]):
  for j in range(y_pred.shape[1]):
      pred.append(np.argmax(y_pred[i][j]))

print(len(pred))
#Converting one hot encoded test label to label
test = []
for i in range(y_tst.shape[0]):
  for j in range(y_tst.shape[1]):
    test.append(np.argmax(y_tst[i][j]))

print(len(test))
from sklearn.metrics import accuracy_score
a = accuracy_score(np.asarray(pred),np.asarray(test))
print('Accuracy is:', a*100)